In [35]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add the src directory to the Python path

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.Template import Template
from src.utils import download_data
from src.mappings import ideology_mapping

In [36]:
template = Template(categories = ['sc'], file_path= '../datasets/test.csv')


Loaded template from ../datasets/test.csv


In [ ]:
df = pd.read_csv('../datasets/Equinox2020.05.2023_MSP.csv', sep = '|')
polity_df = download_data("https://seshat-db.com/api/core/polities/")

Downloaded 839 rows


In [ ]:
df = pd.read_csv('../datasets/mr_dataset.04.2021.csv')
polity_df = download_data("https://seshat-db.com/api/core/polities/")

Downloaded 839 rows


In [41]:
column_mappings = {}
for column in df.columns:
    column_mappings[column] = column.lower().replace('.', '_').replace('-', '_')
column_mappings['Date.From'] = 'year_from'
column_mappings['Date.To'] = 'year_to'
column_mappings['Variable'] = 'name'
df.rename(columns=column_mappings, inplace=True)

In [42]:
df.apply(lambda x: int(polity_df.loc[polity_df['name'] == x['polid'], 'id'].values[0]) if x['polid'] in polity_df['name'].values else print(x.polid), axis=1);

GbEmpir
AkKidar
InEInCo
InBritP
InKanau
InGurjr


In [43]:
df['polity_id'] = df.apply(lambda x: int(polity_df.loc[polity_df['name'] == x['polid'], 'id'].values[0]) if x['polid'] in polity_df['name'].values else None, axis=1)
print(df.polity_id.isna().sum())
df = df[df.polity_id.notna()]
df.polity_id = df.polity_id.astype(int)

df['polity_new_name'] = df.apply(lambda x: polity_df.loc[polity_df['id'] == x['polity_id'], 'new_name'].values[0] if x['polity_id'] in polity_df['id'].values else None, axis=1)

6


In [27]:
print(df.columns)

Index(['nga', 'polid', 'start', 'end', 'dupl', 'primary', 'certain', 'broad',
       'targeted', 'ruler', 'elite', 'commoners', 'afterlife', 'thislife',
       'agency', 'disputed', 'expert', 'infa', 'msp', 'msp_sum', 'minmsp',
       'polity_id', 'polity_new_name'],
      dtype='object')


In [28]:
ideology_mapping['MSP']

{'moral-concern-is-primary': 1,
 'moralizing-enforcement-is-certain': 1,
 'moralizing-norms-are-broad': 1,
 'moralizing-enforcement-is-targeted': 1,
 'moralizing-enforcement-of-rulers': 1,
 'moralizing-religion-adopted-by-elites': 1,
 'moralizing-religion-adopted-by-commoners': 1,
 'moralizing-enforcement-in-afterlife': 1,
 'moralizing-enforcement-in-this-life': 1,
 'moralizing-enforcement-is-agentic': 1}

In [44]:
col_replace = {'primary' : 'moral-concern-is-primary',
               'certain' : 'moralizing-enforcement-is-certain',
               'broad' : 'moralizing-norms-are-broad',
               'targeted' : 'moralizing-enforcement-is-targeted',
               'ruler': 'moralizing-enforcement-of-rulers',
               'elite' : 'moralizing-religion-adopted-by-elites',
               'commoners' : 'moralizing-religion-adopted-by-commoners',
               'afterlife' : 'moralizing-enforcement-in-afterlife',
               'thislife' : 'moralizing-enforcement-in-this-life',
               'agency' : 'moralizing-enforcement-is-agentic',
               'start' : 'year_from',
                'end' : 'year_to',
               }

df.rename(columns=col_replace, inplace=True)

In [45]:
df['is_disputed'] = False
df['is_uncertain'] = False

In [46]:
variables = [col for col in col_replace.values()]

In [47]:

for variable in variables:

    if (variable in ideology_mapping['MSP'].keys()):
        print(variable)
        var_df = df.copy()
        var_df['name'] = variable
        # make sure that if an element of the col variable has more than one entry, the first one is taken
        template.add_to_template(var_df, variable)

moral-concern-is-primary
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the polity's start and end year
Error: The year is outside the 

In [33]:
pols = template.template.PolityName.unique()
msp_pols = df.polity_new_name.unique()

# find polities that are in both datasets
common_pols = set(pols).intersection(set(msp_pols))
print(f"found {len(common_pols)} common polities")
# find polities only in pols
pols_only = set(pols).difference(set(msp_pols))
print(f"found {len(pols_only)} polities only in pols")

# find polities only in pt_pols
msp_pols_only = set(msp_pols).difference(set(pols))
print(f"found {len(msp_pols_only)} polities only in msp_pols")
print(msp_pols_only)

found 339 common polities
found 500 polities only in pols
found 0 polities only in msp_pols
set()


In [50]:
template.save_dataset("../datasets/MSP_template_new.csv")

Saved template to ../datasets/MSP_template_new.csv


In [49]:
template.template

,NGA,PolityID,PolityName,StartYear,EndYear,polity-territories,polity-populations,population-of-the-largest-settlements,settlement-hierarchies,administrative-levels,...,moral-concern-is-primary,moralizing-enforcement-is-certain,moralizing-norms-are-broad,moralizing-enforcement-is-targeted,moralizing-enforcement-of-rulers,moralizing-religion-adopted-by-elites,moralizing-religion-adopted-by-commoners,moralizing-enforcement-in-afterlife,moralizing-enforcement-in-this-life,moralizing-enforcement-is-agentic
0,Southern Mesopotamia,132,iq_abbasid_cal_1,750.0,946.0,"{'t': [[800, 900]], 'value': [[(8300000, 83000...","{'t': [[800, 900]], 'value': [[(23000000, 3300...","{'t': [[800, 900], [800, 900]], 'value': [[(70...","{'t': [[750.0, 946.0]], 'value': [[(6, 6), (6,...","{'t': [[750.0, 946.0]], 'value': [[(6, 6), (6,...",...,"{'t': [[750, 946]], 'value': [[(1.0, 1.0), (1....","{'t': [[750, 946]], 'value': [[(1.0, 1.0), (1....","{'t': [[750, 946]], 'value': [[(1.0, 1.0), (1....","{'t': [[750, 946]], 'value': [[(1.0, 1.0), (1....","{'t': [[750, 946]], 'value': [[(1.0, 1.0), (1....","{'t': [[750, 946]], 'value': [[(1.0, 1.0), (1....","{'t': [[750, 946]], 'value': [[(1.0, 1.0), (1....","{'t': [[750, 946]], 'value': [[(1.0, 1.0), (1....","{'t': [[750, 946]], 'value': [[(1.0, 1.0), (1....","{'t': [[750, 946]], 'value': [[(1.0, 1.0), (1...."
1,Southern Mesopotamia,484,iq_abbasid_cal_2,1191.0,1258.0,"{'t': [[1200]], 'value': [[(750000, 750000)]],...","{'t': [[1200]], 'value': [[(3900000, 3900000)]...","{'t': [[1191.0, 1258.0]], 'value': [[(1000000,...","{'t': [[1191.0, 1258.0]], 'value': [[(4, 5), (...","{'t': [[1191.0, 1258.0]], 'value': [[(5, 5), (...",...,"{'t': [[1191, 1207]], 'value': [[(1.0, 1.0), (...","{'t': [[1191, 1207]], 'value': [[(1.0, 1.0), (...","{'t': [[1191, 1207]], 'value': [[(1.0, 1.0), (...","{'t': [[1191, 1207]], 'value': [[(1.0, 1.0), (...","{'t': [[1191, 1207]], 'value': [[(1.0, 1.0), (...","{'t': [[1191, 1207]], 'value': [[(1.0, 1.0), (...","{'t': [[1191, 1207]], 'value': [[(1.0, 1.0), (...","{'t': [[1191, 1207]], 'value': [[(1.0, 1.0), (...","{'t': [[1191, 1207]], 'value': [[(1.0, 1.0), (...","{'t': [[1191, 1207]], 'value': [[(1.0, 1.0), (..."
2,Susiana,107,ir_achaemenid_emp,-550.0,-331.0,"{'t': [[-539, -501, -500, -451, -450, -331]], ...","{'t': [[-500, -400]], 'value': [[(20000000, 26...","{'t': [[-550.0, -331.0]], 'value': [[(200000, ...","{'t': [[-550.0, -331.0]], 'value': [[(5, 6), (...","{'t': [[-550.0, -331.0]], 'value': [[(6, 8), (...",...,"{'t': [[-539, -331]], 'value': [[(1.0, 1.0), (...","{'t': [[-539, -331]], 'value': [[(1.0, 1.0), (...","{'t': [[-539, -331]], 'value': [[(1.0, 1.0), (...","{'t': [[-539, -331]], 'value': [[(1.0, 1.0), (...","{'t': [[-539, -331]], 'value': [[(1.0, 1.0), (...","{'t': [[-539, -331]], 'value': [[(1.0, 1.0), (...","{'t': [[-539, -331]], 'value': [[(0.0, 0.0), (...","{'t': [[-539, -331]], 'value': [[(1.0, 1.0), (...","{'t': [[-539, -331]], 'value': [[(1.0, 1.0), (...","{'t': [[-539, -331]], 'value': [[(1.0, 1.0), (..."
3,NaN,637,so_adal_sultanate,1375.0,1543.0,NaN,NaN,NaN,NaN,"{'t': [[1375.0, 1543.0]], 'value': [[(4, 7), (...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,872,tn_aghlabid_dyn,800.0,908.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,NaN,227,et_zagwe,1137.0,1269.0,"{'t': [[1137.0, 1269.0]], 'value': [[(40000, 5...",NaN,NaN,NaN,"{'t': [[1137.0, 1269.0]], 'value': [[(3, 5), (...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
835,NaN,231,dz_zayyanid_dyn,1235.0,1509.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
836,NaN,222,tn_zirid_dyn,973.0,1148.0,"{'t': [[973.0, 1148.0]], 'value': [[(500000, 6...","{'t': [[1000, 1100]], 'value': [[(2000000, 200...",NaN,"{'t': [[973.0, 1148.0]], 'value': [[(3, 3), (3...","{'t': [[973.0, 1148.0]], 'value': [[(5, 5), (5...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
837,Orkhon Valley,444,mn_zungharian_emp,16

In [27]:
df['is_disputed'] = df.disputed == 'yes'
df['is_uncertain'] = df.value_note.apply(lambda x: True if x == 'uncertain' else False)

AttributeError: 'DataFrame' object has no attribute 'value_note'

In [7]:
variables = df.name.unique()

for variable in variables:
    var_df = df.loc[df['name'] == variable]
    variable = variable.replace(' ', '-').lower()
    if (variable in ideology_mapping['MSP'].keys()):
        print(variable)
        variable = variable.lower()
        var_df.name = variable
        var_df.rename(columns={'value_from': variable}, inplace=True)
        template.add_to_template(var_df, variable)

moral-concern-is-primary


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moral-concern-is-primary dataset to template
moralizing-enforcement-is-certain


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moralizing-enforcement-is-certain dataset to template
moralizing-norms-are-broad
Added moralizing-norms-are-broad dataset to template
moralizing-enforcement-is-targeted


...

Added moralizing-enforcement-in-afterlife dataset to template
moralizing-enforcement-in-this-life


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moralizing-enforcement-in-this-life dataset to template
moralizing-enforcement-is-agentic


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.name = variable
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_27835/4136551396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.rename(columns={'value_from': variable}, inplace=True)


Added moralizing-enforcement-is-agentic dataset to template


In [8]:
template.save_dataset("../datasets/MSP_template.csv")

Saved template to ../datasets/MSP_template.csv
